In [1]:
%pip install pandas
%pip install spacy
%pip install difflib
%pip install numpy
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement difflib (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for difflib


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [ ]:
import pandas as pd
import spacy
from difflib import get_close_matches
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Descargar el modelo de lenguaje en español
!python -m spacy download es_core_news_md

# Cargar modelo de lenguaje
nlp = spacy.load("es_core_news_md")

# Cargar los archivos Excel
file_ceibal_combine = "../data/Ceibal.xlsx"
file_codeorg_combine = "../data/CodeOrg.xlsx"
file_thetech_combine = "../data/TheTech.xlsx"
file_restantes_combine = "../data/Restantes.xlsx"
file2 = "Enlaces.xlsx"

df_ceibal = pd.read_excel(file_ceibal_combine)
df_codeorg = pd.read_excel(file_codeorg_combine)
df_thetech = pd.read_excel(file_thetech_combine)
df_restantes = pd.read_excel(file_restantes_combine)

df1 = pd.concat([df_ceibal, df_codeorg, df_thetech, df_restantes], ignore_index=True)  # Datos principales combinados
df2 = pd.read_excel(file2, header=None, names=["Name", "URL"])  # Datos con URLs


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     - -------------------------------------- 1.8/42.3 MB 20.2 MB/s eta 0:00:03
     --------- ----------------------------- 10.0/42.3 MB 32.7 MB/s eta 0:00:01
     ---------------- ---------------------- 18.4/42.3 MB 36.2 MB/s eta 0:00:01
     ------------------------- ------------- 27.3/42.3 MB 37.6 MB/s eta 0:00:01
     -------------------------------- ------ 35.7/42.3 MB 38.4 MB/s eta 0:00:01
     --------------------------------------  42.2/42.3 MB 38.9 MB/s eta 0:00:01
     --------------------------------------  42.2/42.3 MB 38.9 MB/s eta 0:00:01
     --------------------------------------  42.2/42.3 MB 38.9 MB/s eta 0:00:01
     --------------------------------------- 42.3/42.3 MB 26.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [3]:
# Pasarlos a string
df1 = df1.astype(str)
df2 = df2.astype(str)

# Comprobar el tipo de nuevo
print(df1.dtypes)
print(df2.dtypes)

Filename            object
Name                object
Resource type       object
Values              object
Key competences     object
Knowledege areas    object
Cc concepts         object
Ct skills           object
Duration            object
School years        object
Languages           object
dtype: object
Name    object
URL     object
dtype: object


In [4]:
# Crear una nueva columna URL en df1
df1["URL"] = ""

# Crear un diccionario basado en los nombres de df2
url_dict = {row["Name"].lower(): row["URL"] for _, row in df2.iterrows()}

def get_semantic_match(target, choices):
    """Encuentra la mejor coincidencia semántica usando embeddings de spaCy."""
    target_doc = nlp(target)
    best_match = None
    best_score = 0.0
    
    for choice in choices:
        choice_doc = nlp(choice)
        similarity = target_doc.similarity(choice_doc)
        if similarity > best_score:
            best_score = similarity
            best_match = choice
    
    return best_match if best_score > 0.7 else None  # Umbral de 0.7 para coincidencias confiables

# Intentar coincidencias exactas basadas en el Filename
for idx, row in df1.iterrows():
    filename = str(row["Filename"]).strip()
    name = str(row["Name"]).strip().lower()
    
    # Buscar coincidencia basada en filename
    for key, url in url_dict.items():
        if filename in url:
            df1.at[idx, "URL"] = url
            break
    
    # Si no encontró por filename, buscar por name (semántico)
    if not df1.at[idx, "URL"]:
        best_match = get_semantic_match(name, url_dict.keys())
        if best_match:
            df1.at[idx, "URL"] = url_dict[best_match]
        else:
            # Si no hay coincidencia semántica, buscar coincidencia basada en caracteres
            match = get_close_matches(name, url_dict.keys(), n=1, cutoff=0.6)
            if match:
                df1.at[idx, "URL"] = url_dict[match[0]]

# Guardar el resultado en un nuevo archivo Excel
output_file = "output.xlsx"
df1.to_excel(output_file, index=False)

print(f"Archivo guardado en {output_file}")

C:\Users\user\AppData\Local\Temp\ipykernel_14136\2413517763.py:15: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = target_doc.similarity(choice_doc)


Archivo guardado en output.xlsx


In [5]:
# Mostrar el resultado
df1.head()

,Filename,Name,Resource type,Values,Key competences,Knowledege areas,Cc concepts,Ct skills,Duration,School years,Languages,URL
0,Tarjetas-Bebras-I.pdf,Desafío Bebras: Retos de Pensamiento Computaci...,Actividad enchufada,Fomento de la creatividad y del espíritu cient...,"Comunicación Lingüística, Competencia Digital,...",Informática y Robótica,"Algoritmia y Programación, Sistemas Informáticos","Abstracción, Descomposición, Patrones, Pensami...",nan,nan,Español,
1,Tarjetas-Bebras-Tanda-2.pdf,Desafío Bebras,Actividad desenchufada,"Pensamiento crítico, Educación para la Salud, ...","Competencia Digital, Competencia Matemática y ...","Informática y Robótica, Matemáticas","Algoritmia y Programación, Análisis de Datos, ...","Pensamiento Algorítmico y Programación, Razona...",nan,"3º ESO, 4º ESO, 1º Bachillerato, 2º Bachillerato",Español,https://pensamientocomputacional.ceibal.edu.uy...
2,Building_ai_education_ori_digital-2.pdf,Construyendo la Inteligencia Artificial para l...,"Actividad enchufada, Curso",Fomento de la creatividad y del espíritu cient...,"Competencia Digital, Competencia Matemática y ...","Informática y Robótica, Matemáticas, Ética, Fi...","Algoritmia y Programación, Análisis de Datos, ...","Abstracción, Descomposición, Patrones, Pensami...",Semana (2-4 sesiones),"Primer Ciclo EP, Segundo Ciclo EP, Tercer Cicl...",English,https://pensamientocomputacional.ceibal.edu.uy...
3,2023-Numero-la-rana_Guia-docentes.pdf,La rana que aprendió a contar,Programación visual,"Fomento de la creatividad, Pensamiento crítico...","Competencia Digital, Competencia Matemática y ...","Informática y Robótica, Matemáticas, Lengua y ...","Algoritmia y Programación, Análisis de Datos","Abstracción, Descomposición, Pensamiento Algor...",2 Meses (15-30 sesiones),"Primer Ciclo EP, Segundo Ciclo EP, Tercer Cicl...",Español,https://pensamientocomputacional.ceibal.edu.uy...
4,2024-Nuestro-Show-Programado-Guia-docentes.pdf,Nuestro Show programado,"Programación visual, Curso",Fomento de la creatividad y del espíritu cient...,"Comunicación Lingüística, Competencia Digital,...","Informática y Robótica, Arte, Lengua y Literatura",Algoritmia y Programación,"Descomposición, Razonamiento Lógico, Patrones,...",Mes (5-15 sesiones),"Segundo Ciclo EP, Tercer Ciclo EP",Español,https://pensamientocomputacional.ceibal.edu.uy...
